### Frequency map generation steps

1. Create date time seried data of events(start and end date for events)
1a. For flood the data of events are manually created 
2. Read the manually created excel sheet with geoboundary code for each event
3. Merge two dataframe and create year wise fequecny plots of geoboundary code 

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

em_data_filepath='emdat_public_2022_08_11_query_uid-IKGfZc.xlsx'

dfb = pd.read_excel(em_data_filepath, engine='openpyxl',skiprows=6,sheet_name = 'emdat data')

dfb_dr=dfb[dfb['Disaster Type']=='Drought']

ea_cntr=['Djibouti', 'Eritrea','Ethiopia','Kenya','Rwanda', 'Somalia','South Sudan','Sudan (the)','Tanzania, United Republic of']

#dfb1_ea=dfb1[dfb1['Country'].str.contains(ea_cntr)]
dfb_dr_ea=dfb_dr[dfb_dr['Country'].isin(ea_cntr)]
#dfb_dr_ea.info()

dfb_dr_ea_time=dfb_dr_ea[['Dis No','Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month','End Day']]
#dfb_dr_ea_time['Start Month'].tolist()
dfb_dr_ea_time['Start Month']=dfb_dr_ea_time['Start Month'].fillna(1.0)
dfb_dr_ea_time['Start Day']=dfb_dr_ea_time['Start Day'].fillna(1.0)

dfb_dr_ea_time['End Month']=dfb_dr_ea_time['End Month'].fillna(12.0)

def last_day_month(row):
    firstday_month = datetime(int(row['End Year']), int(row['End Month']), 1)
    lastday_month = firstday_month + pd.DateOffset(months=1) - pd.DateOffset(days=1)
    lastday_month_str =lastday_month.strftime("%d")
    return lastday_month_str

 
for idx,row in dfb_dr_ea_time.iterrows():
    dfb_dr_ea_time.loc[idx,'End Day'] = last_day_month(row)    
    #dfb_dr_ea_time.loc[idx,'end_day'] = last_day_month(row)    

    
#dfb_dr_ea_time['End Day']=dfb_dr_ea_time['Start Day'].fillna(28.0)


dfb_dr_ea_time['Start Year']=dfb_dr_ea_time["Start Year"].astype(str)
#dfb_dr_ea_time['Start Month']=dfb_dr_ea_time["Start Month"].astype(str)
dfb_dr_ea_time['Start Month']=dfb_dr_ea_time["Start Month"].apply(lambda x: str(int(x)).zfill(2))
dfb_dr_ea_time['End Year']=dfb_dr_ea_time["End Year"].astype(str)
#dfb_dr_ea_time['End Month']=dfb_dr_ea_time["End Month"].astype(str)
dfb_dr_ea_time['End Month']=dfb_dr_ea_time["End Month"].apply(lambda x: str(int(x)).zfill(2))

#pd.date_range('2014-10-10','2016-01-07', freq='MS').strftime("%Y-%m").tolist()

def create_daterange_list(row):
    start_date_str=row['Start Year']+'-'+row['Start Month']+'-'+str(int(row['Start Day']))
    end_date_str=row['End Year']+'-'+row['End Month']+'-'+row['End Day']
    date_str_list=pd.date_range(start_date_str,end_date_str, freq='D').strftime("%Y-%m-%d").tolist()
    return date_str_list


date_str_list_cont=[]
for idx,row in dfb_dr_ea_time.iterrows():
    #dfb_dr_ea_time.at[idx,'date_str_list'] = create_daterange_list(row)
    #print(idx)
    #print(row)
    date_str_list=create_daterange_list(row)
    date_str_list_cont.append(date_str_list)
    
#dfb_dr_ea_time

#df_dr=dfb_dr_ea_time.explode('date_str_list')

#df_dr1=df_dr.drop_duplicates('date_str_list')
dfb_dr_ea_time['date_str_list']=date_str_list_cont
df_dr=dfb_dr_ea_time.explode('date_str_list')
#df_dr1=df_dr.drop_duplicates('date_str_list')

df_t=df_dr.groupby('date_str_list').agg({'Dis No':lambda x: list(x)})

dft1=df_t.reset_index()
dft1['date']=dft1['date_str_list'].apply(pd.to_datetime)
dft1['year']=dft1['date'].dt.year
df=dft1.groupby('year').agg({'Dis No': 'sum'})
df['disno1'] = df['Dis No'].apply(lambda x: list(set(x)))
df_year=df.reset_index()
df_year1=df_year[['year','disno1']]

In [ ]:
filepath='./'

filename=f'{filepath}em_dat_ea_drought_adm2_v2.xlsx'

dfb_dr=pd.read_excel(filename, engine='openpyxl')
#dfb_dr['year']= dfb_dr['Dis No'].str[:4]
#dfb_dr

dfb_dr['adm2_list'] = dfb_dr.geob_adm2_list.apply(lambda x: literal_eval(str(x)))
#dfb_dr['disno_list'] = dfb_dr['Dis No'].apply(lambda x: literal_eval(str(x)))

dfb_dr1=dfb_dr[['Dis No','adm2_list']]